In [1]:
import random
import numpy as np


class Board:
    def __init__(self, n):
        self.n_queen = n
        self.map = [[0 for j in range(n)] for i in range(n)]
        self.encoded = self.encode()
        self.fit = 0

    def set_queens(self):
        for i in range(self.n_queen):
            j = random.randint(0, self.n_queen - 1)
            self.map[i][j] = 1

    def fitness(self):
        self.fit = self.n_queen * (self.n_queen - 1) // 2
        for i in range(self.n_queen):
            for j in range(self.n_queen):
                if self.map[i][j] == 1:
                    for k in range(1, self.n_queen - i):
                        if self.map[i + k][j] == 1:
                            self.fit -= 1
                        if j - k >= 0 and self.map[i + k][j - k] == 1:
                            self.fit -= 1
                        if j + k < self.n_queen and self.map[i + k][j + k] == 1:
                            self.fit -= 1
                            
    def encode(self):
        s = ''
        for row in self.map:
            for i in range(len(row)):
                if row[i] == 1:
                    s += str(i)
                    break
        return s
    
    
    def decode_to_board(self, enc):
        self.map = [[0 for j in range(self.n_queen)] for i in range(self.n_queen)]
        c = 0
        for s in enc:
            self.map[c][int(s)] = 1
            c += 1
            
        self.encoded = enc

    
    def show(self):
        print(np.matrix(self.map))
        print("Fitness: ",  self.fit)

        
if __name__ == '__main__':
    test = Board(5)
    test.set_queens()
    test.fitness()
    test.show()
    print(test.encode())

[[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 0 1 0]
 [0 1 0 0 0]
 [0 0 0 0 1]]
Fitness:  6
01314


In [22]:
class Hill_Climb:
    
    def __init__(self, n):
        self.steps = 0
        self.map = n
        self.q_pos = set()  
    
    def moveQueen(self):
        temp = self.map.map[0]
        self.map.map[0] = [0,0,0,0,0]
        mfit = 0
        mindx = 0
        for i in range(5):
            self.map.map[i] = [0,0,0,0,0]
            mfit = 0
            mindx = 0
            for j in range(5):
                self.map.map[i][j] = 1
                self.map.fitness()
                if(self.map.fit > mfit):
                    mfit = self.map.fit
                    mindx = j
                self.map.map[i][j] = 0
                if(mfit == 10):
                    print("Finished")
                    self.map.fitness()
                    self.map.show()
                    return self
            self.map.map[i][mindx] = 1
        self.moveQueen()
        
if __name__ == '__main__':
    home = Board(5)
    home.set_queens()
    home.fitness()
    home.show()
    print("\n")
    climb = Hill_Climb(home)
    climb.moveQueen()
        
    

[[0 0 0 0 1]
 [1 0 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 1 0]]
Fitness:  6


Finished
[[0 1 0 0 0]
 [0 0 0 0 1]
 [0 0 0 0 0]
 [1 0 0 0 0]
 [0 0 0 1 0]]
Fitness:  10


In [51]:
class Genetic:
    def __init__(self, n):
        self.n = n
        self.steps = 0
        self.states = dict()
        self.changed_states = []

        while len(self.states) < 8:
            board = Board(n)
            board.set_queens()
            board.fitness()
            self.states[board.encode()] = board.fit
            
        

    def selection(self):
        total_fit = 0
        for r in self.states:
            total_fit += self.states[r]
        
        # sorting by fitness
        self.states = {k : v for k, v, in sorted(self.states.items(), key=lambda item: item[1], reverse=True)}
        keys = list(self.states.keys())
        
        selects = dict()
        # calculating selection probabilities
        for r in keys:
            selects[r] = (self.states[r] / total_fit)
        
        # selecting states
        for i in range(8):
            r = np.random.randint(0, 101) / 100
             
            if r < selects[keys[0]]:
                self.changed_states.append(keys[0])
            
            elif r >= selects[keys[0]] and r < selects[keys[1]]:
                self.changed_states.append(keys[1])
                
            elif r >= selects[keys[1]] + selects[keys[0]] and r < selects[keys[2]] + selects[keys[1]] + selects[keys[0]]:
                self.changed_states.append(keys[2])
                
            elif r >= selects[keys[2]] + selects[keys[1]] + selects[keys[0]] and r < selects[keys[3]] + selects[keys[2]] + selects[keys[1]] + selects[keys[0]]:
                self.changed_states.append(keys[3])
                
            elif r >= selects[keys[3]] + selects[keys[2]] + selects[keys[1]] and r < selects[keys[4]] + selects[keys[3]] + selects[keys[2]] + selects[keys[1]] + selects[keys[0]]:
                self.changed_states.append(keys[4])
            
            elif r >= selects[keys[4]] + selects[keys[3]] + selects[keys[2]] + selects[keys[1]] and r < selects[keys[5]] + selects[keys[4]] + selects[keys[3]] + selects[keys[2]] + selects[keys[1]] + selects[keys[0]]:
                self.changed_states.append(keys[5])
        
            elif r >= selects[keys[5]] + selects[keys[4]] + selects[keys[3]] + selects[keys[2]] + selects[keys[1]] and r < selects[keys[6]] + selects[keys[5]] + selects[keys[4]] + selects[keys[3]] + selects[keys[2]] + selects[keys[1]] + selects[keys[0]]:
                self.changed_states.append(keys[6])
                
            else:
                self.changed_states.append(keys[7])
                
        print('Selections', self.changed_states)        
                
    
    # Updates self.changed_states after crossover0
    def crossover(self):

        r1 = np.random.randint(0, len(self.changed_states[0])-1)
        # print('Index 1:', r1)
        new_0 = self.changed_states[0][:r1] + self.changed_states[1][r1:]
        new_1 = self.changed_states[1][:r1] + self.changed_states[0][r1:]
        self.changed_states[0] = new_0
        self.changed_states[1] = new_1
        
        r2 = np.random.randint(0, len(self.changed_states[0])-1)
        # print('Index 2:', r2)
        new_2 = self.changed_states[2][:r2] + self.changed_states[3][r2:]
        new_3 = self.changed_states[3][:r2] + self.changed_states[2][r2:]
        self.changed_states[2] = new_2
        self.changed_states[3] = new_3
        
        r3 = np.random.randint(0, len(self.changed_states[0])-1)
        # print('Index 3:', r3)
        new_4 = self.changed_states[4][:r3] + self.changed_states[5][r3:]
        new_5 = self.changed_states[5][:r3] + self.changed_states[4][r3:]
        self.changed_states[4] = new_4
        self.changed_states[5] = new_5
        
        r4 = np.random.randint(0, len(self.changed_states[0])-1)
        # print('Index 2:', r2)
        new_6 = self.changed_states[6][:r4] + self.changed_states[7][r4:]
        new_7 = self.changed_states[7][:r4] + self.changed_states[6][r4:]
        self.changed_states[6] = new_6
        self.changed_states[7] = new_7

        print('Crossover ', self.changed_states)
    

    
    # Updates self.changed_states after mutation
    def mutate(self):
        
        for i in range(len(self.changed_states)):
            # print('before', self.changed_states[i])
            s = list(self.changed_states[i])
            s[np.random.randint(0, len(s))] = str(np.random.randint(0, len(s)))
            s = ''.join(s)
            self.changed_states[i] = s
            # print('after ', self.changed_states[i], '\n')
            
        print('Mutated   ', self.changed_states)
        
    
    def update_states(self):
        for enc in self.changed_states:
            board = Board(self.n)
            board.decode_to_board(enc)
            board.fitness()
            self.states[board.encode()] = board.fit
            
            
    def get_max_fitness(self):
        max_fit = max(self.states.values())
        return max_fit
    
    def get_solution(self):
        self.states = {k : v for k, v, in sorted(self.states.items(), key=lambda item: item[1], reverse=True)}
        keys = list(self.states.keys())
        return keys[0]
        
        
        
if __name__ == '__main__':
    n = 5
    genetic = Genetic(n)
    trials = np.random.randint(10, 100)
    steps = 0
    for i in range(trials):
        genetic.selection()
        genetic.crossover()
        genetic.mutate()
        genetic.update_states()
        steps += 1
        if genetic.get_max_fitness() == 10:
            print('Optimal Solution found')
            break
            
    print('\n\n---------------------------------\n')
    print('Solution:', genetic.get_solution())
    print('Fitness:', genetic.get_max_fitness())
    print('Steps:', steps)



Selections ['44224', '44224', '23113', '11202', '24420', '11122', '21210', '23113']
Crossover  ['44224', '44224', '11202', '23113', '24422', '11120', '21213', '23110']
Mutated    ['44024', '34224', '11202', '23111', '24423', '13120', '21211', '23111']
Selections ['44024', '34224', '11202', '23111', '24423', '13120', '21211', '23111', '11122', '24420', '24420', '23113', '23113', '23113', '23113', '24420']
Crossover  ['44024', '34224', '23111', '11202', '24420', '13123', '23111', '21211', '11122', '24420', '24420', '23113', '23113', '23113', '23113', '24420']
Mutated    ['44024', '34324', '13111', '11402', '24020', '13143', '43111', '22211', '11122', '20420', '24220', '23110', '23113', '20113', '23112', '24120']
Selections ['44024', '34324', '13111', '11402', '24020', '13143', '43111', '22211', '11122', '20420', '24220', '23110', '23113', '20113', '23112', '24120', '24020', '24020', '24020', '24020', '24420', '24020', '11402', '13120']
Crossover  ['44324', '34024', '11402', '13111', '240